In [ ]:
from pyproj import Proj
from pathlib import Path
from pypism.profiles import extract_profile, add_function, compute_normals, compute_tangentials, distance_from_start
from pypism.interpolation import InterpolationMatrix
import numpy as np
import xarray as xr
import geopandas as gp
from copy import deepcopy

In [ ]:
profiles_path = Path("../tests/data/greenland-flux-gates-29_500m.gpkg")
profiles_gp = gp.read_file(profiles_path)

In [ ]:
obs_file = Path("../tests/data/test_vel_g9000m.nc")
ds = xr.open_dataset(obs_file)
ds = xr.open_dataset(Path("../tests/data/velsurf_mag_gris_g9000m_id_0_0_50.nc"))

In [ ]:
profiles_gp["nx"] = profiles_gp.apply(lambda row: add_function(compute_normals, row["geometry"])[0], axis=1)
profiles_gp["ny"] = profiles_gp.apply(lambda row: add_function(compute_tangentials, row["geometry"])[0], axis=1)
profiles_gp["distance_from_start"] = profiles_gp.apply(lambda row: add_function(distance_from_start, row["geometry"]), axis=1)
for m_var in ds.data_vars:
    if m_var not in ("mapping", "lon_bounds", "lon_bnds", "lat_bounds", "lat_bnds", "time_bounds", "time_bnds"):
        profiles_gp[m_var] = profiles_gp.apply(lambda row: extract_profile(ds[m_var], row)[0], axis=1)

In [ ]:
pism_files = Path("../tests/data").glob("vel*.nc")

In [ ]:
p_gp = deepcopy(profiles_gp)
experiments = []
for experiment_file in pism_files:
    print(experiment_file)
    ds = xr.open_dataset(experiment_file)
    print(ds)
    p_gp["nx"] = p_gp.apply(lambda row: add_function(compute_normals, row["geometry"])[0], axis=1)
    p_gp["ny"] = p_gp.apply(lambda row: add_function(compute_tangentials, row["geometry"])[0], axis=1)
    p_gp["distance_from_start"] = p_gp.apply(lambda row: add_function(distance_from_start, row["geometry"]), axis=1)
    for m_var in ds.data_vars:
        if m_var not in ("mapping", "lon_bounds", "lon_bnds", "lat_bounds", "lat_bnds", "time_bounds", "time_bnds"):
            p_gp[m_var] = p_gp.apply(lambda row: extract_profile(ds[m_var], row)[0], axis=1)
    experiments.append(p_gp)

In [ ]:
experiments